In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
data=pd.read_csv('/content/drive/MyDrive/ml_data/naration_vec.csv')
data.head()

,AC_ID,st_tokens,vectorized,sum_vec,k_cat
0,3644877,"['cash', 'withdraw']","[array([-2.70026140e-02, -3.76681685e-02, 1.7...",[-0.01512962 -0.00533655 0.08984514 0.068006...,[10]
1,1175256,"['cash', 'withdrawal', 'ac']","[array([-2.70026140e-02, -3.76681685e-02, 1.7...",[-0.01192736 -0.06130909 0.15097429 0.086923...,[10]
2,2298226,"['cash', 'withdraw', 'micro', 'merchant', 'poi...","[array([-2.70026140e-02, -3.76681685e-02, 1.7...",[-0.30668834 -0.00592415 0.25022923 0.066398...,[1]
3,4066427,"['credit', 'merchant', 'cash', 'withdraw', 'se...","[array([-0.06479806, 0.05488066, 0.0247791 ,...",[-0.12975151 -0.02486618 0.21576638 0.036840...,[2]
4,38418,"['cash', 'deposit', 'ac']","[array([-2.70026140e-02, -3.76681685e-02, 1.7...",[-0.01582591 -0.08448684 0.09600903 0.077138...,[3]


In [3]:
data['nar']=data['st_tokens'].str.replace('[^A-Za-z\s]', '',regex=True)

In [4]:
data=data.drop('vectorized',axis=1)

In [5]:
data=data.drop('st_tokens',axis=1)

In [6]:
data=data.drop('sum_vec',axis=1)

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize 
stop_words=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
stop_words.add('cash')
# ,'ac','credit','debit','mm'
stop_words.add('ac')
stop_words.add('credit')
stop_words.add('debit')
stop_words.add('mm')

In [9]:
def tokenize(text):
    return [w for w in word_tokenize(text) if not w in stop_words]

In [10]:
data['tok']=data['nar'].apply(tokenize)

In [11]:
data.head()

,AC_ID,k_cat,nar,tok
0,3644877,[10],cash withdraw,[withdraw]
1,1175256,[10],cash withdrawal ac,[withdrawal]
2,2298226,[1],cash withdraw micro merchant point mm ac,"[withdraw, micro, merchant, point]"
3,4066427,[2],credit merchant cash withdraw service ac,"[merchant, withdraw, service]"
4,38418,[3],cash deposit ac,[deposit]


In [12]:
token_list=data['tok'].tolist()
token_list[:10]

[['withdraw'],
 ['withdrawal'],
 ['withdraw', 'micro', 'merchant', 'point'],
 ['merchant', 'withdraw', 'service'],
 ['deposit'],
 ['deposit'],
 ['new', 'atm', 'card', 'charge'],
 ['palli', 'bidyut', 'bill', 'collection'],
 ['agri', 'agent', 'commission'],
 ['inter', 'agent', 'withdraw']]

In [13]:
from gensim.models import Word2Vec

# Assuming 'tokens' is a list of tokenized documents
model = Word2Vec(token_list,min_count=1)
# X = [model[token] for token in tokens]


In [68]:
print(model)

Word2Vec<vocab=38400, vector_size=100, alpha=0.025>


In [14]:
data['vectorized'] = data['tok'].apply(lambda x: [model.wv.get_vector(word ,norm=True) for word in x])

In [15]:
data.tail()

,AC_ID,k_cat,nar,tok,vectorized
128598,4010179,[10],guatala udc,"[guatala, udc]","[[-0.033698983, 0.1603815, 0.022689482, 0.0774..."
128599,4199653,[0],transaction fund transfer smart app regular fu...,"[transaction, fund, transfer, smart, app, regu...","[[-0.036927376, 0.061073553, 0.06834131, -0.14..."
128600,2389876,[3],cash deposit,[deposit],"[[-0.16920891, 0.03277916, -0.0038454516, 0.09..."
128601,4817300,[10],,[],[]
128602,789386,[10],,[],[]


In [16]:
def avg_vec(vectors):
    s_vec=np.zeros(100)
    cnt=0
    for v in vectors:
        cnt+=1
        s_vec+=v
    s_res=s_vec/cnt
    return s_res

In [ ]:
def vec_pr(vecs):
  for v in vecs:
    print(v)

vec_pr(data['vectorized'][2])

In [17]:
data['avg_vec']=data['vectorized'].apply(avg_vec)

In [18]:
data=data.drop('vectorized',axis=1)


In [78]:
data.head()

,AC_ID,k_cat,nar,tok,avg_vec
0,3644877,[10],cash withdraw,"[cash, withdraw]","[0.013483261689543724, 0.028128000209107995, 0..."
1,1175256,[10],cash withdrawal ac,"[cash, withdrawal, ac]","[0.04203909387191137, -0.049564197193831205, 0..."
2,2298226,[1],cash withdraw micro merchant point mm ac,"[cash, withdraw, micro, merchant, point, mm, ac]","[-0.07496801710554532, 0.034611711877265146, 0..."
3,4066427,[2],credit merchant cash withdraw service ac,"[credit, merchant, cash, withdraw, service, ac]","[-0.01592357022066911, -0.019762506242841482, ..."
4,38418,[3],cash deposit ac,"[cash, deposit, ac]","[0.028572234014670055, -0.022196612786501646, ..."


In [82]:
nan_rows = data[data['avg_vec'].isna()]

In [83]:
nan_rows

,AC_ID,k_cat,nar,tok,avg_vec


In [93]:
data.tail()

,AC_ID,k_cat,nar,tok,avg_vec
128598,4010179,[10],guatala udc,"[guatala, udc]","[-0.020193786593154073, 0.1242935098707676, 0...."
128599,4199653,[0],transaction fund transfer smart app regular fu...,"[transaction, fund, transfer, smart, app, regu...","[-0.09358644853769378, 0.11370717234570872, 0...."
128600,2389876,[3],cash deposit,"[cash, deposit]","[-0.05778633430600166, -0.03454362018965185, 0..."
128601,4817300,[10],,[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
128602,789386,[10],,[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."


In [94]:
data = data[~np.isnan(data['avg_vec'].map(lambda arr: np.any(np.isnan(arr))))]

In [87]:
# data = data[~np.array([np.array_equal(arr, np.zeros(100)) for arr in data['avg_vec']])]

In [19]:
data = data[data['avg_vec'].apply(lambda arr: np.all(pd.notna(arr)))] # for deleting bengali rows

In [30]:
sub_data=data[:10000]


In [22]:
sub_data

,AC_ID,k_cat,nar,tok,avg_vec
0,3644877,[10],cash withdraw,[withdraw],"[-0.3215008080005646, 0.18247219920158386, 0.0..."
1,1175256,[10],cash withdrawal ac,[withdrawal],"[-0.14098483324050903, 0.0003409186319913715, ..."
2,2298226,[1],cash withdraw micro merchant point mm ac,"[withdraw, micro, merchant, point]","[-0.015055198222398758, 0.04443533648736775, 0..."
3,4066427,[2],credit merchant cash withdraw service ac,"[merchant, withdraw, service]","[-0.0351947546005249, 0.03402328553299109, 0.1..."
4,38418,[3],cash deposit ac,[deposit],"[-0.16920891404151917, 0.032779160887002945, -..."
...,...,...,...,...,...
999,5650263,[0],smart app fund transfer dp installment purpose...,"[smart, app, fund, transfer, dp, installment, ...","[0.029029711957264226, 0.049569783033803105, 0..."
1000,738234,[10],agent bipulashar,"[agent, bipulashar]","[-0.179119773209095, 0.12092044577002525, 0.10..."
1001,5255179,[10],agent balance transfer,"[agent, balance, transfer]","[-0.04401805500189463, 0.06879523334403832, 0...."
1002,3346328,[2],jiku rikuswapnatandra nishita sumon,"[jiku, rikuswapnatandra, nishita, sumon]","[-0.09180832747370005, 0.11843772744759917, 0...."


In [31]:
avg_vec_list=sub_data['avg_vec'].tolist()
# avg_vec_list[:2]
type(avg_vec_list)

list

In [24]:
len(avg_vec_list)

1000

In [101]:
avg_vec_list[128600:]

[]

In [54]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=.25, min_samples=25).fit_predict(avg_vec_list)

In [55]:
dbscan.shape

(10000,)

In [56]:
np.unique(dbscan)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [57]:
sub_data['d_cat']=dbscan

<ipython-input-57-4f0b50fe52e8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_data['d_cat']=dbscan


In [58]:
sub_data

,AC_ID,k_cat,nar,tok,avg_vec,d_cat
0,3644877,[10],cash withdraw,[withdraw],"[-0.3215008080005646, 0.18247219920158386, 0.0...",-1
1,1175256,[10],cash withdrawal ac,[withdrawal],"[-0.14098483324050903, 0.0003409186319913715, ...",-1
2,2298226,[1],cash withdraw micro merchant point mm ac,"[withdraw, micro, merchant, point]","[-0.015055198222398758, 0.04443533648736775, 0...",-1
3,4066427,[2],credit merchant cash withdraw service ac,"[merchant, withdraw, service]","[-0.0351947546005249, 0.03402328553299109, 0.1...",-1
4,38418,[3],cash deposit ac,[deposit],"[-0.16920891404151917, 0.032779160887002945, -...",0
...,...,...,...,...,...,...
10057,4044420,[10],bissho bahi,"[bissho, bahi]","[-0.054924534633755684, 0.15501704812049866, 0...",-1
10058,720243,[3],cash deposit siam,"[deposit, siam]","[-0.15006566792726517, 0.11278467811644077, -0...",2
10059,39678,[0],smart app fund transfer family expense purpose,"[smart, app, fund, transfer, family, expense, ...","[0.02817127254093066, 0.0507479440420866, 0.11...",4
10060,172575,[2],withdraw md tajul islam ac,"[withdraw, md, tajul, islam]","[-0.2634422034025192, 0.07664136774837971, 0.0...",2


In [36]:
def ds_pred(vec):
#     p = kmeans.predict(vec.reshape(1,-1))
      # print(type(vec))
      p=dbscan.fit_predict(vec.reshape(1,-1))
      return p

In [37]:
print(ds_pred(sub_data['avg_vec'][3]))

AttributeError: ignored